# Getting familiar with the <span style="color:blue">Transit For London</span> API: PART 2

This notebook only contains the final task of the project and answer the question: <strong>Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster?</strong>

In [1]:
import requests as re
from IPython.display import JSON

### Adjusted custom get request function for the Transportation For London API 
Prints response code and returns JSON tuple: regular and Jupyter-formatted

Set base url for all API calls

In [2]:
# Custom get function 
def get(url, endpoint):
    res = re.get(url + endpoint)
    json_r = res.json()
    json_f = JSON(json_r)
    
    print(res.status_code)
    
    return json_r, json_f

# Set the base url for all future get requests

base ='https://api.tfl.gov.uk/'

### After looking at the documentation, I wrote a long string with the search endpoint for the required trip by <span style="color:blue">the Tube</span> from Heathrow to Tower Bridge

In [3]:
edpt = 'Journey/JourneyResults/Heathrow%20Airport/to/Tower%20Bridge?timeIs=Arriving&journeyPreference=LeastInterchange&mode=Tube'\
'&accessibilityPreference=NoRequirements&walkingSpeed=Slow&cyclePreference=None&bikeProficiency=Easy'

In [4]:
journey = get(base, endpoint=edpt)

300


### Note: code <span style="color:blue">300</span> means there were multiple responses to request. I took a look at the keys to dictionary.

In [5]:
print(journey[0].keys())

dict_keys(['$type', 'toLocationDisambiguation', 'fromLocationDisambiguation', 'viaLocationDisambiguation', 'recommendedMaxAgeMinutes', 'searchCriteria', 'journeyVector'])


### I examined the <span style="color:blue">formatted JSON</span> to know the request keys better to see how both <span style="color:red">toLocationDisambiguation</span> and <span style="color:red">fromLocationDisambiguation</span> are structured.

In [6]:
journey[1]

<IPython.core.display.JSON object>

### After examining the dictionary, I decided to make a list of <span style="color:blue">common place names</span> near the <span style="color:red">Tower of London</span> as there are multiple places for the destination. I did the same for the departure point at the <span style="color:red">airport</span>.

In [7]:
tower_vicinity = []
for location in journey[0]['toLocationDisambiguation']['disambiguationOptions']:
    tower_vicinity.append(location['place']['commonName'])
tower_vicinity

['City of London, Tower Bridge',
 'Southwark, One Tower Bridge',
 'Wapping (London), Tower Bridge Quay',
 'London Bridge, Tower Bridge Road',
 'Southwark, Tower Bridge Road',
 'Harrow (London), Tower Bridge Mews',
 'Southwark, Think Tower Bridge',
 'City of London, Tower Bridge Approach',
 'Tower Hamlets, Tower Bridge Exhibition',
 'London Bridge, Tower Bridge / City Hall',
 'Southwark, Draft House Tower Bridge',
 'Tower Hamlets [Stepney], Grange Tower Bridge Hotel',
 'City of Westminster, Novotel Tower Bridge Hotel',
 'Tower Hamlets [Stepney], Try Tag Rugby Tower Bridge',
 'Southwark, Tower Bridge Magistrates Court',
 'Southwark, Chicken Cottage Takeaway Food, Tower Bridge']

In [8]:
airport = []
for location in journey[0]['fromLocationDisambiguation']['disambiguationOptions']:
    airport.append(location['place']['commonName'])
airport

['Hillingdon, Heathrow Airport Terminal 2',
 'Heathrow Airport, Heathrow Airport Terminal 3',
 'Hillingdon, London Heathrow Airport Terminal 5',
 'Hillingdon, Heathrow Airport Terminal 5 Arrivals',
 'Hillingdon, Heathrow Airport Terminal 4 Arrivals',
 'Hillingdon, Heathrow Airport Terminal 3 Arrivals',
 'Hillingdon, United Airlines, Heathrow Airport Terminal 2',
 'Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station']

### I made a dictionary with each place (departure and arrival) and the latitude and longitude of each location

In [9]:
tower_vicinity_ll = {}
for location in journey[0]['toLocationDisambiguation']['disambiguationOptions']:
    tower_vicinity_ll[location['place']['commonName']] = (location['place']['lat'],location['place']['lon'])
print(tower_vicinity_ll)

{'City of London, Tower Bridge': (51.50599630145, -0.07502752220999999), 'Southwark, One Tower Bridge': (51.503958003700006, -0.07740449013999999), 'Wapping (London), Tower Bridge Quay': (51.50622947685, -0.07334614114), 'London Bridge, Tower Bridge Road': (51.50227892501, -0.0786566734), 'Southwark, Tower Bridge Road': (51.49743849977, -0.08015688257), 'Harrow (London), Tower Bridge Mews': (51.55796833405, -0.3351140985), 'Southwark, Think Tower Bridge': (51.500226935220006, -0.07910317811999999), 'City of London, Tower Bridge Approach': (51.508139082110006, -0.07408698432000001), 'Tower Hamlets, Tower Bridge Exhibition': (51.50551623644, -0.07536476862000001), 'London Bridge, Tower Bridge / City Hall': (51.50354961675, -0.07662917924), 'Southwark, Draft House Tower Bridge': (51.502917014030004, -0.07698722055), 'Tower Hamlets [Stepney], Grange Tower Bridge Hotel': (51.511481522249994, -0.07227427369), 'City of Westminster, Novotel Tower Bridge Hotel': (51.51092054902, -0.0775437131),

In [10]:
airport_ll = {}
for location in journey[0]['fromLocationDisambiguation']['disambiguationOptions']:
    airport_ll[location['place']['commonName']] = (location['place']['lat'],location['place']['lon'])
print(airport_ll)

{'Hillingdon, Heathrow Airport Terminal 2': (51.46962511264, -0.44968473098), 'Heathrow Airport, Heathrow Airport Terminal 3': (51.47254839367, -0.45724573875), 'Hillingdon, London Heathrow Airport Terminal 5': (51.47238603683, -0.48688352825), 'Hillingdon, Heathrow Airport Terminal 5 Arrivals': (51.47212537413, -0.48689215164), 'Hillingdon, Heathrow Airport Terminal 4 Arrivals': (51.46046306549, -0.44636767138), 'Hillingdon, Heathrow Airport Terminal 3 Arrivals': (51.471544550269996, -0.45886341771), 'Hillingdon, United Airlines, Heathrow Airport Terminal 2': (51.4697660699, -0.44946398635999996), 'Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station': (51.47198549429, -0.45478818469000004)}


### I made a function for quick display to get the correct index quickly

In [11]:
def place_index(place_list):
    for i, place in enumerate(place_list):
        print(f'[{i}] -> {place}')

In [12]:
place_index(tower_vicinity)

[0] -> City of London, Tower Bridge
[1] -> Southwark, One Tower Bridge
[2] -> Wapping (London), Tower Bridge Quay
[3] -> London Bridge, Tower Bridge Road
[4] -> Southwark, Tower Bridge Road
[5] -> Harrow (London), Tower Bridge Mews
[6] -> Southwark, Think Tower Bridge
[7] -> City of London, Tower Bridge Approach
[8] -> Tower Hamlets, Tower Bridge Exhibition
[9] -> London Bridge, Tower Bridge / City Hall
[10] -> Southwark, Draft House Tower Bridge
[11] -> Tower Hamlets [Stepney], Grange Tower Bridge Hotel
[12] -> City of Westminster, Novotel Tower Bridge Hotel
[13] -> Tower Hamlets [Stepney], Try Tag Rugby Tower Bridge
[14] -> Southwark, Tower Bridge Magistrates Court
[15] -> Southwark, Chicken Cottage Takeaway Food, Tower Bridge


## After looking at various locations in the tower vicinity, I decided to make my presentation from the perspective of the <span style="color:blue">Novotel Tower Bridge Hotel</span>
- <span style="font-size:20px"> the <span style="color:blue">business case</span> of the hotel working with the <span style="color:blue">City of London</span> in an initiative to use more public transportation</span>

### Confirm the hotel name and location

In [13]:
novotel_tower_bridge = journey[0]['toLocationDisambiguation']['disambiguationOptions'][12]['place']
name = novotel_tower_bridge['commonName']
lat = novotel_tower_bridge['lat']
long = novotel_tower_bridge['lon']
print(name)
print(f'lat: {lat}, long: {long}')

City of Westminster, Novotel Tower Bridge Hotel
lat: 51.51092054902, long: -0.0775437131


In [14]:
place_index(airport)

[0] -> Hillingdon, Heathrow Airport Terminal 2
[1] -> Heathrow Airport, Heathrow Airport Terminal 3
[2] -> Hillingdon, London Heathrow Airport Terminal 5
[3] -> Hillingdon, Heathrow Airport Terminal 5 Arrivals
[4] -> Hillingdon, Heathrow Airport Terminal 4 Arrivals
[5] -> Hillingdon, Heathrow Airport Terminal 3 Arrivals
[6] -> Hillingdon, United Airlines, Heathrow Airport Terminal 2
[7] -> Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station


## After looking at Heathrow Airport on Google Maps, I decided the basic departure point from the airport to be the <span style="color:blue">Rail Station</span> at the final index, and at <span style="color:blue">Terminal 5</span>, which is in separte locations and a bit of a distance from the terminals 1 and 3 (terminal 1-3 are in the centre, terminal 4 is in the south, and terminal 5 in the west side of the airport)

# From Terminals 2 & 3 Rail Station

### Get the GPS coordinates for Novotel Tower Bridge Hotel and Airport Rail Station to use in my URL request

In [15]:
novotel = tower_vicinity[12]
terminal = airport[7]
lat_1 = str(airport_ll[terminal][0])
long_1 = str(airport_ll[terminal][1])

lat_2 = str(tower_vicinity_ll[novotel][0])
long_2 = str(tower_vicinity_ll[novotel][1])
lat_1, long_1, lat_2, long_2

('51.47198549429', '-0.45478818469000004', '51.51092054902', '-0.0775437131')

### Build the URL manually to use in custom function below

In [16]:
url='https://api.tfl.gov.uk/Journey/JourneyResults/51.47198549429%2C-0.45478818469000004/to/51.51092054902%2C-0.0775437131?'

### Two custom functions: one to get the possilbe journeys, and one to calculate the time of the journeys

In [43]:
# default the url from Heathrow Airport Terminals 2 & 3 Rail Station to Novotel Tower Bridge Hotel
def get_to_novotel(url, date, time , going, transport):
    pref = 'LeastTime'
    res = re.get(url, params={'date': date, 'time': time, 'timeIs': going, 'mode': transport, 'journeyPreference': pref})
    json_r = res.json()
    json_f = JSON(json_r)
    print(res.status_code)
    return json_r, json_f

def journey_times(journeys):
    average = 0
    for journey in journeys:
        time = journey['duration']
        average += time
        print(f'{time} minutes')
    print(f'Average journey time: {round(average/len(journeys), 2)} minutes')

### I checked the trip times in the morning, at noon, and in the evening to check for conistency. I did this for both the Tube and the bus.

In [18]:
morning_tube = get_to_novotel(url, '20220607', '0900', 'Departing', 'tube') 
morning_tube[1]

200


<IPython.core.display.JSON object>

In [19]:
noon_tube = get_to_novotel(url, '20220607', '1200', 'Departing', 'tube') 
noon_tube[1]

200


<IPython.core.display.JSON object>

In [20]:
evening_tube = get_to_novotel(url, '20220607', '1800', 'Departing', 'tube') 
evening_tube[1]

200


<IPython.core.display.JSON object>

In [44]:
journeys = morning_tube[0]['journeys']
journey_times(journeys)
journeys = noon_tube[0]['journeys']
journey_times(journeys)
journeys = evening_tube[0]['journeys']
journey_times(journeys)

75 minutes
74 minutes
75 minutes
Average journey time: 74.67 minutes
74 minutes
74 minutes
77 minutes
Average journey time: 75.0 minutes
76 minutes
77 minutes
76 minutes
Average journey time: 76.33 minutes


In [22]:
morning_bus = get_to_novotel(url, '20220607', '0900', 'Departing', 'bus')
morning_bus[1]

200


<IPython.core.display.JSON object>

In [23]:
noon_bus = get_to_novotel(url, '20220607', '1200', 'Departing', 'bus')
noon_bus[1]

200


<IPython.core.display.JSON object>

In [24]:
evening_bus = get_to_novotel(url, '20220607', '1200', 'Departing', 'bus')
evening_bus[1]

200


<IPython.core.display.JSON object>

In [45]:
journeys = morning_bus[0]['journeys']
journey_times(journeys)
journeys = noon_bus[0]['journeys']
journey_times(journeys)
journeys = evening_bus[0]['journeys']
journey_times(journeys)

152 minutes
175 minutes
173 minutes
Average journey time: 166.67 minutes
173 minutes
152 minutes
178 minutes
Average journey time: 167.67 minutes
173 minutes
152 minutes
178 minutes
Average journey time: 167.67 minutes


### A custom function for showing the directions when taking the Tube

In [50]:
# This function is only good for the Tube
def directions(route):
    start = route['journeys'][0]['startDateTime'][-8:-3]   
    stop = route['journeys'][0]['arrivalDateTime'][-8:-3] 
    duration = route['journeys'][0]['duration']
    print(f'Your start time from the Terminals 1-3 is: {start}.' )
    print(f'Your arrival time at the Novotel London Tower Bridge is: {stop}')
    print(f'Duration: {duration} minutes')
    print('\nDirections:')
    print(route['journeys'][0]['legs'][0]['instruction']['detailed'])
    print(route['journeys'][0]['legs'][1]['instruction']['detailed'])
    departure = route['journeys'][0]['legs'][1]['departureTime'][-8:-3]
    print(f'\t -> Train departs at {departure}')
    print(route['journeys'][0]['legs'][2]['instruction']['detailed'])
    departure = route['journeys'][0]['legs'][2]['departureTime'][-8:-3]
    print(f'\t -> Train departs at {departure}')
    print(route['journeys'][0]['legs'][3]['instruction']['detailed'])
    
# directions(route)    

In [51]:
morning_route =  morning_tube[0]
directions(morning_route)

Your start time from the Terminals 1-3 is: 08:59.
Your arrival time at the Novotel London Tower Bridge is: 10:14
Duration: 75 minutes

Directions:
Walk to Heathrow Terminals 2 & 3 Underground Station
Piccadilly line towards Arnos Grove
	 -> Train departs at 09:07
District line towards Upminster
	 -> Train departs at 09:39
Walk to 25 Savage Gardens, City


In [28]:
noon_route =  noon_tube[0]
directions(noon_route)

Your start time from the Terminals 1-3 is: 11:59.
Your arrival time at the Novotel London Tower Bridge is: 13:13
Duration: 74 minutes

Directions:
Walk to Heathrow Terminals 2 & 3 Underground Station
Piccadilly line towards Cockfosters -> Train departs at 12:07
District line towards Upminster -> Train departs at 12:39
Walk to 25 Savage Gardens, City


# From Terminal 5 Arrivals

### Build the URL using two custom functions: can be used with any other departure and arrival points

In [29]:
def departure_arrival_gps(departure_index, arrival_index):
    departure_point = airport[departure_index]
    destination = tower_vicinity[arrival_index]
    
    lat_departure = str(airport_ll[departure_point][0])
    long_departure = str(airport_ll[departure_point][1])

    lat_arrival = str(tower_vicinity_ll[destination][0])
    long_arrival = str(tower_vicinity_ll[destination][1])
    
    return lat_departure, long_departure, lat_arrival, long_arrival

def build_url(base_url, endpoint, departure_index, arrival_index):
    lat_1, long_1, lat_2, long_2 = departure_arrival_gps(departure_index, arrival_index)
    
    ll_ep = lat_1 + '%2C' + long_1 + '/to/' + lat_2 + '%2C' + long_2 + '?'
    final_url = base + endpoint + ll_ep
    
    return final_url

In [30]:
place_index(airport)

[0] -> Hillingdon, Heathrow Airport Terminal 2
[1] -> Heathrow Airport, Heathrow Airport Terminal 3
[2] -> Hillingdon, London Heathrow Airport Terminal 5
[3] -> Hillingdon, Heathrow Airport Terminal 5 Arrivals
[4] -> Hillingdon, Heathrow Airport Terminal 4 Arrivals
[5] -> Hillingdon, Heathrow Airport Terminal 3 Arrivals
[6] -> Hillingdon, United Airlines, Heathrow Airport Terminal 2
[7] -> Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station


### Use index 3 (Heathrow Terminal 5 Arrivals) and 12 (Nototel Tower Bridge)

In [31]:
default_url = build_url(base_url='https://api.tfl.gov.uk/', endpoint='Journey/JourneyResults/', departure_index=3, arrival_index=12)
# Confrim URL is correcdt
print(default_url)

https://api.tfl.gov.uk/Journey/JourneyResults/51.47212537413%2C-0.48689215164/to/51.51092054902%2C-0.0775437131?


### New custom function defaults the URL from Terminal 5 arrivals

In [32]:
# default the url from Heathrow Airport Terminals 5 Arrivals as built above to Novotel Tower Bridge Hotel
def get_to_novotel_5(date, time , going, transport, url=default_url):
    pref = 'LeastTime'
    res = re.get(url, params={'date': date, 'time': time, 'timeIs': going, 'mode': transport, 'journeyPreference': pref})
    json_r = res.json()
    json_f = JSON(json_r)
    print(res.status_code)
    return json_r, json_f

In [33]:
morning_tube_5 = get_to_novotel_5('20220607', '0900', 'Departing', 'tube')
morning_tube_5[1]

200


<IPython.core.display.JSON object>

In [34]:
afternoon_tube_5 = get_to_novotel_5('20220607', '1300', 'Departing', 'tube')
afternoon_tube_5[1]

200


<IPython.core.display.JSON object>

In [35]:
evening_tube_5 = get_to_novotel_5('20220607', '1800', 'Departing', 'tube')
evening_tube_5[1]

200


<IPython.core.display.JSON object>

In [52]:
journey_times(morning_tube_5[0]['journeys'])
journey_times(afternoon_tube_5[0]['journeys'])
journey_times(evening_tube_5[0]['journeys'])

75 minutes
75 minutes
74 minutes
Average journey time: 74.67 minutes
74 minutes
77 minutes
74 minutes
Average journey time: 75.0 minutes
76 minutes
76 minutes
77 minutes
Average journey time: 76.33 minutes


In [37]:
morning_bus_5 = get_to_novotel_5('20220607', '0900', 'Departing', 'bus')
morning_bus_5[1]

200


<IPython.core.display.JSON object>

In [38]:
afternoon_bus_5 = get_to_novotel_5('20220607', '1300', 'Departing', 'bus')
afternoon_bus_5[1]

200


<IPython.core.display.JSON object>

In [39]:
evening_bus_5 = get_to_novotel_5('20220607', '1800', 'Departing', 'bus')
evening_bus_5[1]

200


<IPython.core.display.JSON object>

In [53]:
journey_times(morning_bus_5[0]['journeys'])
journey_times(afternoon_bus_5[0]['journeys'])
journey_times(evening_bus_5[0]['journeys'])

192 minutes
146 minutes
175 minutes
Average journey time: 171.0 minutes
178 minutes
185 minutes
176 minutes
Average journey time: 179.67 minutes
176 minutes
167 minutes
133 minutes
Average journey time: 158.67 minutes


### New custom directions function that can be used for both the Tube and the bus (or other modes of transport)

In [55]:
def directions_5(route):
    route = route['journeys'][0]
    start = route['startDateTime'][-8:-3]  
    stop = route['arrivalDateTime'] [-8:-3]
    print(f'Your start time from the Terminal 5 is: {start}' )
    print(f'Your arrival time at the Novotel London Tower Bridge is: {stop}')
    print('\nDirections:')
    steps = len(route['legs'])
    for i in range(steps):
        print(str(i+1) + ") ", end="")
        
        if i == 0:
            print("Walk to " + str(route['legs'][i]['arrivalPoint']['commonName'])) 
        elif (i != 0) and (i != steps - 1):
            print("Take ", end="")
            print(route['legs'][i]['instruction']['summary'])
        else:
            print(route['legs'][i]['instruction']['summary'])
    print(str(steps + 1) + ") Arrive at Novotel London Tower Bridge (25 Savage Gardens)")
        
    fare = route['fare']['totalCost']
    duration = route['duration']

    print(f'\nThe total trip time is approximately: {duration} minutes')
    print(f'\nThe cost of this trip is: £{fare} ')

In [56]:
morning_route_5 = morning_tube_5[0]
directions_5(morning_route_5)

Your start time from the Terminal 5 is: 08:59
Your arrival time at the Novotel London Tower Bridge is: 10:14

Directions:
1) Walk to Heathrow Terminal 5 Underground Station
2) Take Piccadilly line to Hammersmith (Dist&Picc Line)
3) Take District line to Tower Hill
4) Walk to 25 Savage Gardens, City
5) Arrive at Novotel London Tower Bridge (25 Savage Gardens)

The total trip time is approximately: 75 minutes

The cost of this trip is: £550 


In [58]:
afternoon_route_5 = afternoon_tube_5[0]
directions_5(afternoon_route_5)

Your start time from the Terminal 5 is: 12:59
Your arrival time at the Novotel London Tower Bridge is: 14:13

Directions:
1) Walk to Heathrow Terminal 5 Underground Station
2) Take Piccadilly line to Hammersmith (Dist&Picc Line)
3) Take District line to Tower Hill
4) Walk to 25 Savage Gardens, City
5) Arrive at Novotel London Tower Bridge (25 Savage Gardens)

The total trip time is approximately: 74 minutes

The cost of this trip is: £350 


In [59]:
afternoon_route_5 = afternoon_bus_5[0]
directions_5(afternoon_route_5)

Your start time from the Terminal 5 is: 12:58
Your arrival time at the Novotel London Tower Bridge is: 15:56

Directions:
1) Walk to Heathrow Terminal 5
2) Take 482 bus to Hounslow West Station / Vicarage Farm Road
3) Take H91 bus to Hammersmith Bus Station
4) Take 9 bus to Southampton Street / Covent Garden
5) Take 15 bus to Great Tower Street
6) Walk to 25 Savage Gardens, City
7) Arrive at Novotel London Tower Bridge (25 Savage Gardens)

The total trip time is approximately: 178 minutes

The cost of this trip is: £330 


In [67]:
# directions(afternoon_bus_5[0])

In [61]:
noon_bus = get_to_novotel(url, '20220607', '1200', 'Departing', 'bus')
noon_bus[1]

200


<IPython.core.display.JSON object>

In [62]:
directions_5(noon_bus[0])

Your start time from the Terminal 5 is: 11:39
Your arrival time at the Novotel London Tower Bridge is: 14:32

Directions:
1) Walk to Heathrow Central Bus Stn (Rail-Air)
2) Take X26 bus to Broad Street / Teddington
3) Take 33 bus to Manor Road / North Sheen Station
4) Take 337 bus to Clapham Junction Station
5) Take 156 bus to Vauxhall Bus Station
6) Take 196 bus to Elephant & Castle Station
7) Take 35 bus to Gracechurch St / Monument Stn
8) Walk to 25 Savage Gardens, City
9) Arrive at Novotel London Tower Bridge (25 Savage Gardens)

The total trip time is approximately: 173 minutes

The cost of this trip is: £330 


#### Let's check what other modes of transport are available from the airport

In [63]:
modes = get(base, endpoint='Line/Meta/Modes')

200


In [65]:
types = []
other_types = []
for mode in modes[0]: 
    name = mode['modeName']
    if mode['isTflService']:
        types.append(name)
    else:
        other_types.append(mode['modeName'])

In [66]:
for i in types:
    a = get_to_novotel(url, '20220607', '1500', 'Departing', i)
    print(i)

200
bus
404
cable-car
200
cycle-hire
404
dlr
404
elizabeth-line
404
overground
404
replacement-bus
404
river-bus
200
river-tour
404
tflrail
404
tram
200
tube


In [68]:
morning_river_tour = get_to_novotel(url, '20220607', '0900', 'Departing', 'river-tour') 
morning_river_tour[1]

200


<IPython.core.display.JSON object>

In [69]:
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [70]:
arrival = datetime.strptime(morning_river_tour[0]['journeys'][0]['arrivalDateTime'], '%Y-%m-%dT%H:%M:%S')
start = datetime.strptime(morning_river_tour[0]['journeys'][0]['startDateTime'], '%Y-%m-%dT%H:%M:%S')
time = arrival - start
time.seconds/60

75.0

In [71]:
directions_5(morning_river_tour[0])

Your start time from the Terminal 5 is: 08:59
Your arrival time at the Novotel London Tower Bridge is: 10:14

Directions:
1) Walk to Heathrow Terminals 2 & 3 Underground Station
2) Take Piccadilly line to Hammersmith (Dist&Picc Line)
3) Take District line to Tower Hill
4) Walk to 25 Savage Gardens, City
5) Arrive at Novotel London Tower Bridge (25 Savage Gardens)

The total trip time is approximately: 75 minutes

The cost of this trip is: £550 


In [ ]:
# terminal_2 = get(endpoint='Journey/JourneyResults/51.46962511264%2C-0.44968473098/to/51.50599630145%2C-0.07502752220999999?date=20211125&time=0900&timeIs=Departing')

In [ ]:
terminal_2[2]

In [ ]:
JSON(t2.json())

In [ ]:
t3 = re.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.46962511264%2C-0.44968473098/to/51.50599630145%2C-0.07502752220999999?mode=tube')

In [ ]:
JSON(t3.json())